In [1]:
# Importación de librería
# -------------------------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de Pandas
# -------------------------------------------------------------------------------------------------------
pd.set_option('display.max_columns', None) # Para visualizar todas las columnas de los DataFrame
sns.set(style="whitegrid", palette="muted", font_scale=1.1) # Configuración del estilo de los gráficos

In [2]:
# Dataset principal (campaña de marketing)
# -------------------------------------------------------------------------------------------------------
bank = pd.read_csv("bank-additional.csv", decimal=",", index_col= 0)

bank.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_
0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,2-agosto-2019,41.495,-71.233,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.994,-36.4,NaN,5191.0,no,14-septiembre-2016,34.601,-83.923,e9d37224-cb6f-4942-98d7-46672963d097
2,37.0,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,93.994,-36.4,4.857,5191.0,no,15-febrero-2019,34.939,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,93.994,-36.4,NaN,5191.0,no,29-noviembre-2015,49.041,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e
4,56.0,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,93.994,-36.4,NaN,5191.0,no,29-enero-2017,38.033,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


In [3]:
# Detectar variables binarias (solo 0 y 1) y cambiarlas a Si/No
# -------------------------------------------------------------------------------------------------------
binarias = ['default', 'housing', 'loan']
for col in binarias:
    bank[col] = pd.to_numeric(bank[col], errors="coerce").astype("Int64") # Convertir a numérico (maneja '0.0', '1.0' y NaN)
    bank[col] = bank[col].map({1: "Sí", 0: "No"})  # Crear versión categórica Sí/No (NaN se conserva)

In [4]:
# Con el resultado de información general se detecta que 'age' no se ha cargado correctamente
# -------------------------------------------------------------------------------------------------------

# 1. Se limpia strings y se convierte a numérico
bank['age'] = pd.to_numeric(bank['age'], errors="coerce")

# 2. Quitar decimales inncesarios convirtiendo a entero
bank['age'] = bank['age'].astype('Int64')


print("Tipo de dato:", bank["age"].dtype)
print("Nulos en age:", bank["age"].isnull().sum())
print(bank["age"].describe())

Tipo de dato: Int64
Nulos en age: 5120
count      37880.0
mean     39.977112
std      10.437957
min           17.0
25%           32.0
50%           38.0
75%           47.0
max           98.0
Name: age, dtype: Float64


In [5]:
# Corregir variable que aparece como categórica cuando tendría que ser numérica
# -------------------------------------------------------------------------------------------------------
bank["emp.var.rate"] = pd.to_numeric(bank["emp.var.rate"], errors="coerce")

In [6]:
# Detectar todas las columnas categóricas (objecto string)
# -------------------------------------------------------------------------------------------------------
categoricas = bank.select_dtypes(include=["object"]).columns

# Filtramos solo las que tienen NaN
# -------------------------------------------------------------------------------------------------------
categoricas_con_nan = [col for col in categoricas if bank[col].isna().sum() > 0]

print("Columnas categorizad con NaN detectadas:", categoricas_con_nan)

# Reemplazar NaN por "Desconocido"
# -------------------------------------------------------------------------------------------------------
for col in categoricas_con_nan:
    bank[col] = bank[col].fillna("Desconocido")

Columnas categorizad con NaN detectadas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'date']


In [7]:
# Comprobamos que no haya ningún duplicado
# -------------------------------------------------------------------------------------------------------
bank.duplicated().sum()

# Eliminamos los posibles duplicados que haya
# -------------------------------------------------------------------------------------------------------
bank.drop_duplicates(inplace=True)

In [8]:
# Eliminamos las columnas que no nos aprontan datos para el análisis
# -------------------------------------------------------------------------------------------------------
bank = bank.drop(columns=['nr.employed', 'latitude', 'longitude'])

In [9]:
# Conversión de fechas del dataset bank
# -------------------------------------------------------------------------------------------------------
meses = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04", "mayo": "05", "junio": "06", 
    "julio": "07", "agosto": "08", "septiembre": "09", "setiembre": "09", "octubre": "10", 
    "noviembre": "11", "diciembre": "12"
}

def convertir_fecha(fecha_str):
    """Convierte fechas con meses en español o inglés a datetime"""
    fecha = str(fecha_str).lower()

    for mes, num in meses.items(): # Reemplazar meses en español por números
        if mes in fecha:
            fecha = fecha.replace(mes, num)
            return pd.to_datetime(fecha, format="%d-%m-%Y", errors="coerce")
    
    return pd.to_datetime(fecha, dayfirst=True, errors="coerce") # Si no hay mes en español, intentar parseo directo

if "date" in bank.columns:
    bank["date"] = bank["date"].apply(convertir_fecha)


bank[["date"]].head()

,date
0,2019-08-02
1,2016-09-14
2,2019-02-15
3,2015-11-29
4,2017-01-29


In [10]:
inconsistencies = []

# === REGLA 1: Edad ===
# No puede haber clientes con menos de 18 años o más de 100.
mask = (bank["age"] < 18) | (bank["age"] > 100)
for i in bank.index[mask]:
    inconsistencies.append((i, "Edad fuera de rango (<18 o >100)"))

# === REGLA 2: Duración de llamada ===
# La duración no puede ser negativa o igual a 0.
mask = bank["duration"] <= 0
for i in bank.index[mask]:
    inconsistencies.append((i, "Duración de llamada <= 0"))

# === REGLA 3: Número de contactos en la campaña ===
# campaign debe ser al menos 1 y nunca negativo.
mask = bank["campaign"] <= 0
for i in bank.index[mask]:
    inconsistencies.append((i, "Campaign <= 0"))

# === REGLA 4: Días desde el último contacto ===
# En este dataset 999 significa “nunca contactado antes”.
# Si aparece cualquier otro valor raro (ej. <0), lo marcamos como inconsistencia.
mask = (bank["pdays"] < 0) & (bank["pdays"] != 999)
for i in bank.index[mask]:
    inconsistencies.append((i, "pdays < 0 y != 999"))

# === REGLA 5: Contactos anteriores ===
# previous no puede ser negativo.
mask = bank["previous"] < 0
for i in bank.index[mask]:
    inconsistencies.append((i, "Previous < 0"))

# === REGLA 6: Consistencia entre previous y poutcome ===
# Si previous = 0, entonces poutcome debería ser 'nonexistent'.
mask = (bank["previous"] == 0) & (bank["poutcome"] != "nonexistent")
for i in bank.index[mask]:
    inconsistencies.append((i, "Previous=0 pero poutcome != nonexistent"))

# --- Crear DataFrame de inconsistencias ---
inconsistencies_df = pd.DataFrame(inconsistencies, columns=["Index", "Inconsistency"])

# --- Ver cuántas inconsistencias por regla ---
print(inconsistencies_df["Inconsistency"].value_counts())

# --- Filtrar filas inconsistentes ---
inconsistent_indices = inconsistencies_df["Index"].unique()
df_inconsistent_rows = bank.loc[inconsistent_indices]

df_inconsistent_rows.head()


Inconsistency
Previous=0 pero poutcome != nonexistent    37103
Edad fuera de rango (<18 o >100)               5
Duración de llamada <= 0                       4
Name: count, dtype: int64


,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y,date,id_
37140,17,student,SINGLE,Desconocido,No,Sí,No,cellular,432,3,4,2,SUCCESS,-2.9,92.201,-31.4,NaN,no,2015-12-04,e022180e-6a12-45c8-94ca-f97d3933264c
37539,17,student,SINGLE,basic.9y,No,Sí,No,cellular,182,2,999,2,FAILURE,-2.9,92.201,-31.4,NaN,no,2018-08-26,a1e3729b-2f4a-4784-bb34-ac9f20b30183
37558,17,student,SINGLE,basic.9y,No,Sí,No,cellular,92,3,4,2,SUCCESS,-2.9,92.201,-31.4,NaN,no,2016-12-09,d3a215d5-7238-4c3c-878a-86f8639143c7
37579,17,student,SINGLE,basic.9y,No,Desconocido,Desconocido,cellular,498,2,999,1,FAILURE,-2.9,92.201,-31.4,0.869,yes,2018-12-08,42337e9a-8b2b-48b8-b6e1-cdb94e16c282
38274,17,student,SINGLE,Desconocido,No,No,Sí,cellular,896,1,2,2,SUCCESS,-3.4,92.431,-26.9,0.742,yes,2015-02-24,a6dcee6b-d8a5-4c96-8abf-2d3a05b5b29a


In [11]:
# 1. Corregimos si previous = 0, entonces poutcome = 'nonexistent' 
bank.loc[bank['previous'] == 0, "poutcome"] = "nonexistent"

# 2. Eliminar edades imposibles (<18 o >100)
bank = bank[(bank['age'] >= 18) & (bank["age"] <= 100)]

# 3. Eliminar llamadas con duración <= 0
bank = bank[bank['duration'] > 0]

In [12]:
print("Filas finales en bank:", bank.shape[0])

Filas finales en bank: 37871


In [13]:
# Comprobamos que ya no hay inconsistencias

# Edad fuera de rango
print("Edades fuera de rango:", bank[(bank["age"] < 18) | (bank["age"] > 100)].shape[0])

# Duración <= 0
print("Duración <= 0:", bank[bank["duration"] <= 0].shape[0])

# Previous=0 pero poutcome != nonexistent
check_incons = bank[(bank["previous"] == 0) & (bank["poutcome"] != "nonexistent")]
print("Previous=0 y poutcome != nonexistent:", check_incons.shape[0])

Edades fuera de rango: 0
Duración <= 0: 0
Previous=0 y poutcome != nonexistent: 0


In [14]:
bank.to_csv('bank_clean.csv')